# Please upvote if you like my work :)

<h2>Imports</h2>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import progressbar
import nltk
import matplotlib.pyplot as plt
import re

from tqdm.auto import tqdm
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h2>Upload Datasets</h2>

In [ ]:
TRAIN_DATA_PATH = "/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv"
TEST_DATA_PATH = "/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"
VALID_DATA_PATH = "../input/jigsaw-toxic-severity-rating/validation_data.csv"
SAMPLE_SUBMISSION = "/kaggle/input/jigsaw-toxic-severity-rating/sample_submission.csv"

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)
df_validation_data = pd.read_csv(VALID_DATA_PATH)
df_sample_submission = pd.read_csv(SAMPLE_SUBMISSION)

# Scoring training data

In [ ]:
# for col in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
#     print(f'****** {col} *******')
#     display(df_train.loc[df_train[col]==1,['comment_text',col]].sample(10))

In [ ]:
# Create a score that messure how much toxic is a comment
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

# cat_mtpl = {'obscene': 3, 'toxic': 4, 'threat': 4, 
#             'insult': 2, 'severe_toxic': 4, 'identity_hate': 2}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].sum(axis=1)

df_train['y'] = df_train['score']

min_len = (df_train['y'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['y'] == 0].sample(n=min_len, random_state=201)  # take non toxic comments
df_train_new = pd.concat([df_train[df_train['y'] > 0], df_y0_undersample])  # make new df
df_train_new

<h2>Clean text</h2>

Both data sets (train al test) need to be cleaned for better predictions.

In [ ]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+')  # Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml')  # Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) # Remove special Charecters
    text = re.sub(' +', ' ', text) # Remove Extra Spaces
    text = text.strip().lower() # remove spaces at the beginning and at the end of string and make string lower
    
    # lemmatization
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split(' ')])
    # del stopwords
    text = ' '.join([word for word in text.split(' ') if word not in stop])

    return text

In [ ]:
def clean(data, col):
    
    data[col] = data[col].str.replace('https?://\S+|www\.\S+', ' social medium ')      
        
    data[col] = data[col].str.lower()
    data[col] = data[col].str.replace("4", "a") 
    data[col] = data[col].str.replace("2", "l")
    data[col] = data[col].str.replace("5", "s") 
    data[col] = data[col].str.replace("1", "i") 
    data[col] = data[col].str.replace("!", "i") 
    data[col] = data[col].str.replace("|", "i") 
    data[col] = data[col].str.replace("0", "o") 
    data[col] = data[col].str.replace("l3", "b") 
    data[col] = data[col].str.replace("7", "t") 
    data[col] = data[col].str.replace("7", "+") 
    data[col] = data[col].str.replace("8", "ate") 
    data[col] = data[col].str.replace("3", "e") 
    data[col] = data[col].str.replace("9", "g")
    data[col] = data[col].str.replace("6", "g")
    data[col] = data[col].str.replace("@", "a")
    data[col] = data[col].str.replace("$", "s")
    data[col] = data[col].str.replace("#ofc", " of fuckin course ")
    data[col] = data[col].str.replace("fggt", " faggot ")
    data[col] = data[col].str.replace("your", " your ")
    data[col] = data[col].str.replace("self", " self ")
    data[col] = data[col].str.replace("cuntbag", " cunt bag ")
    data[col] = data[col].str.replace("fartchina", " fart china ")    
    data[col] = data[col].str.replace("youi", " you i ")
    data[col] = data[col].str.replace("cunti", " cunt i ")
    data[col] = data[col].str.replace("sucki", " suck i ")
    data[col] = data[col].str.replace("pagedelete", " page delete ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("i'm", " i am ")
    data[col] = data[col].str.replace("offuck", " of fuck ")
    data[col] = data[col].str.replace("centraliststupid", " central ist stupid ")
    data[col] = data[col].str.replace("hitleri", " hitler i ")
    data[col] = data[col].str.replace("i've", " i have ")
    data[col] = data[col].str.replace("i'll", " sick ")
    data[col] = data[col].str.replace("fuck", " fuck ")
    data[col] = data[col].str.replace("f u c k", " fuck ")
    data[col] = data[col].str.replace("shit", " shit ")
    data[col] = data[col].str.replace("bunksteve", " bunk steve ")
    data[col] = data[col].str.replace('wikipedia', ' social medium ')
    data[col] = data[col].str.replace("faggot", " faggot ")
    data[col] = data[col].str.replace("delanoy", " delanoy ")
    data[col] = data[col].str.replace("jewish", " jewish ")
    data[col] = data[col].str.replace("sexsex", " sex ")
    data[col] = data[col].str.replace("allii", " all ii ")
    data[col] = data[col].str.replace("i'd", " i had ")
    data[col] = data[col].str.replace("'s", " is ")
    data[col] = data[col].str.replace("youbollocks", " you bollocks ")
    data[col] = data[col].str.replace("dick", " dick ")
    data[col] = data[col].str.replace("cuntsi", " cuntsi ")
    data[col] = data[col].str.replace("mothjer", " mother ")
    data[col] = data[col].str.replace("cuntfranks", " cunt ")
    data[col] = data[col].str.replace("ullmann", " jewish ")
    data[col] = data[col].str.replace("mr.", " mister ")
    data[col] = data[col].str.replace("aidsaids", " aids ")
    data[col] = data[col].str.replace("njgw", " nigger ")
    data[col] = data[col].str.replace("wiki", " social medium ")
    data[col] = data[col].str.replace("administrator", " admin ")
    data[col] = data[col].str.replace("gamaliel", " jewish ")
    data[col] = data[col].str.replace("rvv", " vanadalism ")
    data[col] = data[col].str.replace("admins", " admin ")
    data[col] = data[col].str.replace("pensnsnniensnsn", " penis ")
    data[col] = data[col].str.replace("pneis", " penis ")
    data[col] = data[col].str.replace("pennnis", " penis ")
    data[col] = data[col].str.replace("pov.", " point of view ")
    data[col] = data[col].str.replace("vandalising", " vandalism ")
    data[col] = data[col].str.replace("cock", " dick ")
    data[col] = data[col].str.replace("asshole", " asshole ")
    data[col] = data[col].str.replace("youi", " you ")
    data[col] = data[col].str.replace("afd", " all fucking day ")
    data[col] = data[col].str.replace("sockpuppets", " sockpuppetry ")
    data[col] = data[col].str.replace("iiprick", " iprick ")
    data[col] = data[col].str.replace("penisi", " penis ")
    data[col] = data[col].str.replace("warrior", " warrior ")
    data[col] = data[col].str.replace("loil", " laughing out insanely loud ")
    data[col] = data[col].str.replace("vandalise", " vanadalism ")
    data[col] = data[col].str.replace("helli", " helli ")
    data[col] = data[col].str.replace("lunchablesi", " lunchablesi ")
    data[col] = data[col].str.replace("special", " special ")
    data[col] = data[col].str.replace("ilol", " i lol ")
    data[col] = data[col].str.replace(r'\b[uU]\b', 'you')
    data[col] = data[col].str.replace(r"what's", "what is ")
    data[col] = data[col].str.replace(r"\'s", " is ")
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace('\s+', ' ')  # will remove more than one whitespace character
#     text = re.sub(r'\b([^\W\d_]+)(\s+\1)+\b', r'\1', re.sub(r'\W+', ' ', text).strip(), flags=re.I)  # remove repeating words coming immediately one after another
    data[col] = data[col].str.replace(r'(.)\1+', r'\1\1') # 2 or more characters are replaced by 2 characters
#     text = re.sub(r'((\b\w+\b.{1,2}\w+\b)+).+\1', r'\1', text, flags = re.I)
    data[col] = data[col].str.replace("[:|♣|'|§|♠|*|/|?|=|%|&|-|#|•|~|^|>|<|►|_]", '')
    
    
    data[col] = data[col].str.replace(r"what's", "what is ")    
    data[col] = data[col].str.replace(r"\'ve", " have ")
    data[col] = data[col].str.replace(r"can't", "cannot ")
    data[col] = data[col].str.replace(r"n't", " not ")
    data[col] = data[col].str.replace(r"i'm", "i am ")
    data[col] = data[col].str.replace(r"\'re", " are ")
    data[col] = data[col].str.replace(r"\'d", " would ")
    data[col] = data[col].str.replace(r"\'ll", " will ")
    data[col] = data[col].str.replace(r"\'scuse", " excuse ")
    data[col] = data[col].str.replace(r"\'s", " ")

    # Clean some punctutations
    data[col] = data[col].str.replace('\n', ' \n ')
    data[col] = data[col].str.replace(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3')
    # Replace repeating characters more than 3 times to length of 3
    data[col] = data[col].str.replace(r'([*!?\'])\1\1{2,}',r'\1\1\1')    
    # Add space around repeating characters
    data[col] = data[col].str.replace(r'([*!?\']+)',r' \1 ')    
    # patterns with repeating characters 
    data[col] = data[col].str.replace(r'([a-zA-Z])\1{2,}\b',r'\1\1')
    data[col] = data[col].str.replace(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1')
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].str.replace(r'[ ]{2,}',' ').str.strip()   
    data[col] = data[col].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    return data

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


lemmatizer = WordNetLemmatizer()
stop = stopwords.words('english')

# tqdm.pandas()
# df_train_new['clean_text'] = df_train_new['comment_text'].progress_apply(text_cleaning)
df_train_new = clean(df_train_new, 'comment_text')

In [ ]:
tqdm.pandas()
df_train_new['clean_text'] = df_train_new['comment_text'].progress_apply(text_cleaning)

# Compare few models

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgbm
from xgboost import XGBRegressor, XGBClassifier
from sklearn.linear_model import Ridge, LogisticRegression, RidgeCV, ElasticNet, SGDRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor, VotingRegressor

In [ ]:
# df_train_new = df_train_new.reset_index(drop=True)
labels = df_train_new['y']
comments = df_train_new['clean_text']

vectorizer = TfidfVectorizer(min_df=3, max_df=0.5, analyzer='char_wb', ngram_range=(3,5))
comments_tr = vectorizer.fit_transform(comments)
comments_tr

In [ ]:
# vectorizer = TfidfVectorizer()
# counts = vectorizer.fit_transform(X_train)

# regressor = Ridge(alpha=0.2)
# regressor.fit(comments_tr, labels)

# models = [('ridge05', Ridge(random_state=42, alpha=0.8)),
# #           ('linSVR', LinearSVR(random_state=42)),
#           ('sgd', SGDRegressor(random_state=42))]

# final_estimator = lgbm.LGBMRegressor(random_state=42)

# regressor = VotingRegressor(estimators=models)
regressor = Ridge(random_state=42, alpha=0.8)
regressor.fit(comments_tr, labels)

## Lets test our model

In [ ]:
# preprocess val data

tqdm.pandas()
df_validation_data = clean(df_validation_data, 'less_toxic')
df_validation_data = clean(df_validation_data, 'more_toxic')
df_validation_data['less_toxic'] = df_validation_data['less_toxic'].progress_apply(text_cleaning)
df_validation_data['more_toxic'] = df_validation_data['more_toxic'].progress_apply(text_cleaning)

less_toxic = vectorizer.transform(df_validation_data['less_toxic'])
more_toxic = vectorizer.transform(df_validation_data['more_toxic'])

# make predictions
y_pred_less = regressor.predict(less_toxic)
y_pred_more = regressor.predict(more_toxic)

(y_pred_less < y_pred_more).mean()

0.6765643682742128 - Ridge, TFIDF, words lower  
0.6770293609671848 - Ridge, TFIDF, words lower, severe_toxic=1.5  
0.6786900491563704 - LinearSVR, TFIDF, words lower, severe_toxic=1.5. 0.821 на тесте  
0.6777932775342101 - Ridge, TFIDF, words lower, severe_toxic=1.5, alpha=1.342857142857143

In [ ]:
# regressor = Ridge(alpha=0.8)
# regressor.fit(comments_tr, labels)

# y_pred_less = regressor.predict(less_toxic)
# y_pred_more = regressor.predict(more_toxic)

# print(0.8, (y_pred_less < y_pred_more).mean())

In [ ]:
# for alpha in np.linspace(0.1, 2.5, 25):
#     regressor = Ridge(alpha=alpha)
#     regressor.fit(comments_tr, labels)

#     y_pred_less = regressor.predict(less_toxic)
#     y_pred_more = regressor.predict(more_toxic)

#     print(alpha, (y_pred_less < y_pred_more).mean())

# Predictions and load submission.csv

In [ ]:
# df_test['text'] = df_test['text'].apply(lambda x: ' '.join([word for word in words_pattern.findall(x) if word not in stopwords]))
# df_test['text'] = df_test['text'].apply(lambda elem: elem.strip().lower())
df_test = clean(df_test, 'text')
df_test['text'] = df_test['text'].progress_apply(text_cleaning)
df_test

In [ ]:
df_test['prediction'] = regressor.predict(vectorizer.transform(df_test['text']))
df_test = df_test[['comment_id','prediction']]

In [ ]:
df_test['score'] = df_test['prediction']
df_test = df_test[['comment_id','score']]

In [ ]:
df_test.to_csv('./submission.csv', index=False)

### Please upvote if you like my work :)